# Author: Tim Harmling
- **Description:** Creates a Dataset that can be used from our Handwriting Model with *.jpg and *.txt files. The *.txt files contain the text of the handwritten text in the image. The *.jpg files contain the cropped images of the handwritten text.

# Create Dataset

In [ ]:
import os
import xml.etree.ElementTree as ET
from builtins import print

import tensorflow as tf
# import keras_cv

from tqdm.auto import tqdm
from tensorflow import keras
#from keras_cv import bounding_box
#from keras_cv import visualization
from keras.models import load_model
import numpy as np
from keras.models import Sequential, model_from_json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from keras.callbacks import EarlyStopping
import re

In [ ]:
path = "dataset/xml"

# Get all XML file paths in path_annot and sort them
xml_files = sorted(
    [
        os.path.join(path, file_name)
        for file_name in os.listdir(path)
        if file_name.endswith(".xml")
    ]
)
 
# Get all JPEG image file paths in path_images and sort them
jpg_files = sorted(
    [
        os.path.join(path, file_name)
        for file_name in os.listdir(path)
        if file_name.endswith(".jpg")
    ]
)

In [ ]:
def create_box(bbox):
    xmin = float(bbox.find("xmin").text)
    ymin = float(bbox.find("ymin").text)
    xmax = float(bbox.find("xmax").text)
    ymax = float(bbox.find("ymax").text)
    return [xmin, ymin, xmax, ymax]

In [ ]:
def parse_annotation_fake(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = f'dataset/images/{image_name}'

    boxes = []
    classes = []
    main_classes = []
    sub_classes = []
    main_boxes = []
    sub_boxes = []
    values = []
    
    for obj in root.iter("object"):
        cls = obj.find("name").text
        classes.append(cls)
        
        bbox = obj.find("bndbox")
        boxes.append(create_box(bbox))
    
    
        attributes = obj.findall("attributes/attribute")
        for attribute in attributes:
            value_element = attribute.find("value")
            value = value_element.text
            if value is None:
                continue
            elif value is None or value.lower() in ["true", "false"]:
                continue
            
            # Versuche, den Wert in einen Float zu konvertieren
            try:
                float_value = float(value)
                int_value = int(float_value)
                values.append(str(int_value))  # Hier wird der Integer in einen String umgewandelt
            except ValueError:
                # Wenn die Konvertierung fehlschlägt, füge den originalen Wert zur Liste hinzu
                values.append(value)
            break
        
        bbox = obj.find("bndbox")
        sub_boxes.append(create_box(bbox))
        sub_classes.append(cls)
    return image_path, sub_boxes, sub_classes, values

In [ ]:
from collections import namedtuple
ImageInfo = namedtuple('ImageInfo', ['path', 'boxes', 'values', 'classes'])
image_list = []
for xml_file in tqdm(xml_files): 
    image_path, boxes, classes, values = parse_annotation_fake(xml_file)
    image_list.append(ImageInfo(image_path, boxes, values, classes))
 


### Crop ROI

In [ ]:
import cv2
def crop(xmin, ymin, xmax, ymax, image_path, crop_left_percent, crop_bottom_percent):
    image = cv2.imread(image_path)
    crop_left = int((xmax-xmin) * crop_left_percent)
    crop_bottom = int((ymax-ymin) * crop_bottom_percent)

    cropped_image = image[int(ymin):int(ymax)-crop_bottom, int(xmin)+crop_left:int(xmax)]
    if cropped_image.size == 0:
        print("Cropped image is empty")
        return None
    return cropped_image


In [ ]:
save_path_crops = "dataset/dataset_training"

In [43]:
cropping_params = {
    "ad_erzieher_name":         {"left": 0.1, "bottom": 0},
    "ad_erzieher_vorname":      {"left": 0.125, "bottom": 0},
    "ad_erzieher_tel":          {"left": 0.2, "bottom": 0},
    "ad_erzieher_email":        {"left": 0.1, "bottom": 0},
    "schueler_name":            {"left": 0.1, "bottom": 0},
    "schueler_vorname":         {"left": 0.15, "bottom": 0},
    "schueler_klasse":          {"left": 0.125, "bottom": 0},
    "ad_neue_ad_str_haus_nr":   {"left": 0.275, "bottom": 0},
    "ad_neue_ad_plz":           {"left": 0.175, "bottom": 0},
    "ad_neue_ad_stadt":         {"left": 0.1, "bottom": 0},
    "ad_schueler_datum":        {"left": 0.2, "bottom": 0},
    "ag_auswahl_wahl_1":        {"left": 0.15, "bottom": 0},
    "ag_auswahl_wahl_2":        {"left": 0.15, "bottom": 0},
    "ag_auswahl_wahl_3":        {"left": 0.15, "bottom": 0},
    "ag_schueler_datum":        {"left": 0.3, "bottom": 0},
    #"ad_schueler_unterschrift": {"left": 0.2, "bottom": 0.4}, 
    #"ag_schueler_unterschrift": {"left": 0.2, "bottom": 0.4}, 
}

In [44]:
for index, image in enumerate(image_list):
    boxes = image.boxes
    for i, box in enumerate(boxes):
        xmin, ymin, xmax, ymax = np.array(box)
        crop_left_percent = 0
        crop_bottom_percent = 0
        if image.classes[i] in cropping_params:
            params = cropping_params[image.classes[i]]
            crop_left_percent = float(params["left"])
            crop_bottom_percent = float(params["bottom"])
        else:
            continue

        imgCropped = crop(xmin, ymin, xmax, ymax, image.path, crop_left_percent, crop_bottom_percent)
        if imgCropped is not None:
            if image.values[i] == "0":
                continue
            img_file_path = f"{save_path_crops}/{index}_{i}.jpg"
            txt_file_path = f"{save_path_crops}/{index}_{i}.txt"
            cv2.imwrite(img_file_path, imgCropped)
            with open(txt_file_path, 'w', encoding='utf-8') as file:
                image.values[i] = image.values[i].replace(" ", "|")
                file.write(image.values[i])
                print(f"{image.classes[i]}: {image.values[i]}")


ad_erzieher_name: Lüdtke
ad_erzieher_vorname: Markus
ad_erzieher_tel: 3366806
ad_erzieher_email: luedtke@gmx.de
schueler_name: Lüdtke
schueler_vorname: Mila
schueler_klasse: 6b
ad_neue_ad_str_haus_nr: Schumannstrasse|10
ad_neue_ad_plz: 28213
ad_neue_ad_stadt: Bremen
ad_schueler_datum: 15.04.2024
ad_schueler_datum: 17.04.2024
ad_neue_ad_stadt: Bremen
ad_neue_ad_plz: 28309
ad_neue_ad_str_haus_nr: Marschstrasse|2
ad_erzieher_email: merschristian@gmx.de
ad_erzieher_tel: 4349733
ad_erzieher_vorname: Christian
ad_erzieher_name: Mersmann
schueler_name: Mersmann
schueler_vorname: Finn
schueler_klasse: 6B
schueler_klasse: 6b
schueler_vorname: Luca
schueler_name: Fabisch
ad_erzieher_name: Fabisch
ad_erzieher_vorname: Timo
ad_erzieher_tel: 9383071
ad_erzieher_email: fabisch@gmx.de
ad_neue_ad_str_haus_nr: Orchideenweg|12
ad_neue_ad_plz: 28219
ad_neue_ad_stadt: Bremen
ad_schueler_datum: 18.04.2024
ad_erzieher_name: Jacob
ad_erzieher_vorname: Timo
schueler_name: Jacob
schueler_vorname: Irena
schuele